# (1) Libraries and files Section

In [1]:
#Importing all the libraries that we are going to need in this assignment

#Pandas

import pandas as pd
from pandas import read_csv
import csv
pd.set_option('display.max_rows', None)


#Numpy
import numpy  as np


#Uncomment these lines if you want to se only 2 digits after comma

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
np.seterr(invalid='ignore')

#Matplotlib

import matplotlib
import matplotlib.pyplot as plt

#Sklearn

import sklearn
from sklearn import preprocessing 

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

#math

from math import sqrt

#timeit

import timeit

#warnings

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Convert words into numbers methods
from sklearn.feature_extraction.text import CountVectorizer              #BoW
from sklearn.feature_extraction.text import TfidfVectorizer              #tf-idf

from sklearn.model_selection import train_test_split, cross_val_score    #To divide our data evenly
from sklearn import preprocessing                                        #Create numeric categories using the LabelEncoder and fit-trasnsform pipeline
from sklearn.model_selection import KFold                                #For multiclasses, through loop, instead of cross_val
from sklearn.preprocessing import label_binarize

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
from bs4 import BeautifulSoup
import en_core_web_sm
nlp = en_core_web_sm.load()

The given dataset is in this [link](https://drive.google.com/u/0/uc?id=1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79&export=download)

# (2) Functions

In [2]:
def Word_Vectorizer(X_train, Y_train, X_test, Y_test, method):
    
    le = preprocessing.LabelEncoder()
    le.fit(Y_train)
    
    #Y_train and Y_test form now and then , will have numeric values instead of strings
    #Same for X_train and X_test

    Y_train = le.transform(Y_train)
    Y_test  = le.transform(Y_test)

    vectorizer = method()

    X_train = vectorizer.fit_transform(X_train)
    X_test  = vectorizer.transform(X_test)

    return X_train, Y_train, X_test, Y_test

#function to create the matrix X (m x n) and the y column vector. 
def get_dataset(train_set):

  columns=[]
  for row in train_set.iterrows():
    for column in row:
      # x=column
      columns.append(column)
  
  X_matrix = np.array([])
  # vector = np.array([])
  y_features = np.array([])
  X_matrix = ([1,1,1,1,1,1,1,1])
  
  for i in columns[1::2]:
    vector = np.array([])
    vector = np.append(vector,i)
  
    y_features = np.append(y_features,vector[-1])
    vector = np.delete(vector,-1)
    X_matrix = np.vstack([X_matrix, vector])

  X_matrix = np.delete(X_matrix,  [0] , axis=0)
  y_features = np.array([y_features]).T

  return X_matrix, y_features

def cleanCSV(tweets):
  
  df = tweets[['target','text']]
  df.shape
  columns_titles = ["text","target"]
  df=df.reindex(columns=columns_titles)
  df.columns = ['text', 'sentiment']
  df.shape
  df.loc[df["sentiment"]==4 , "sentiment"] = 'positive'
  df.loc[df['sentiment']==0 , "sentiment"] = 'negative'
  df.shape
  return df

def RemoveSpecialCharacters(x):

  cleanString = re.sub('\W+',' ', x )
  cleanString= ' '.join(cleanString.split())
  return cleanString
 

# (3) Loading and previewing the given dataset

In [3]:
#Will read the necessary .csv files

LocationCSV =r'/content/SentimentTweets.csv'                  

tweets = pd.read_csv(LocationCSV, low_memory=False, quoting=csv.QUOTE_NONE, encoding='utf-8',  error_bad_lines=False)                             #Read csv file

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# (4) Preprocessing and cleaning the data

In [4]:
tweets = cleanCSV(tweets)
tweets.head()

,text,sentiment
0,#brokenpromises...,negative
1,David Carradine so sad. Thai's law not sure i...,negative
2,A @ 415 B @ 425. Tell your bro i say congrats!,positive
3,@littlefluffycat Indeed.,positive
4,Completed Race 4 Life in 58mins with girlies f...,positive


In [5]:
#Converting the text to lowercase

tweets['text'] = tweets['text'].apply(lambda x: str(x).lower())
tweets['text'] = tweets['text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text().strip())        #Removing HTML Tags

In [6]:
tweets.head(10)
tweets['text'] = tweets['text'].apply(lambda x: RemoveSpecialCharacters(x))

In [7]:
X = tweets['text']
y = tweets['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42, stratify=y)

# (5) BoW & Tf-Idf Methods.

In [8]:
from sklearn.metrics import classification_report
import re
from bs4 import BeautifulSoup
import en_core_web_sm
nlp = en_core_web_sm.load()
VecMethods = [CountVectorizer, TfidfVectorizer]

def print_Scores(Y_pred,Y_test):

    print("Accuracy for Logistic Regression Method: ,   "  ,accuracy_score(Y_test,Y_pred))
    print("Precision for Logistic Regression Method:,   " ,precision_score(Y_test,Y_pred, average='weighted'))
    print("Recall for Logistic Regression Method:   ,   ", recall_score(Y_test,Y_pred, average='weighted'))
    print("F1 for Logistic Regression Method:       ,   ", f1_score(Y_test,Y_pred, average='weighted') , "\n\n")

def printMethod(i):

  if(i==0):
    return "BoW"
  else:
    return "Tf-idf"

clf_lr = LogisticRegression(max_iter=1000)

for i in range(len(VecMethods)):

  X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42, stratify=y)
  print("For vectorizing method:", printMethod(i),", we have the follows:\n \n")
  X_train, Y_train, X_test, Y_test = Word_Vectorizer(X_train, y_train, X_test, y_test, VecMethods[i])

  clf_lr.fit(X_train,y_train)
  y_test_pred = clf_lr.predict(X_test)
  print_Scores(y_test_pred,y_test)
  print(classification_report(y_test,y_test_pred))

For vectorizing method: BoW , we have the follows:
 

Accuracy for Logistic Regression Method: ,    0.7973808949947778
Precision for Logistic Regression Method:,    0.7975425271439927
Recall for Logistic Regression Method:   ,    0.7973808949947778
F1 for Logistic Regression Method:       ,    0.7973914964828311 


              precision    recall  f1-score   support

    negative       0.81      0.79      0.80     31583
    positive       0.79      0.80      0.80     30652

    accuracy                           0.80     62235
   macro avg       0.80      0.80      0.80     62235
weighted avg       0.80      0.80      0.80     62235

For vectorizing method: Tf-idf , we have the follows:
 

Accuracy for Logistic Regression Method: ,    0.8001446131597976
Precision for Logistic Regression Method:,    0.8001762300011308
Recall for Logistic Regression Method:   ,    0.8001446131597976
F1 for Logistic Regression Method:       ,    0.8001521580646012 


              precision    recall  f

For **tf-idf** we can see a slightly better behaviour, than the **BoW**. This dataset can be much more better, if we apply *feature scaling* , *feature engineering* and maybe some *standardization*.